# 3) Mutability — Exercises

**Learning goals:** reference semantics, aliasing, in-place vs copy, shallow vs deep copy, default mutable args, safe patterns.

### Warm-ups

1. **In-place or copy?** — Return a **new** list with x appended; don’t mutate input.

```python
def append_copy(xs, x):
    ...
a = [1,2]; b = append_copy(a, 3)
assert a == [1,2] and b == [1,2,3]
```

2. **Safe default parameter**

```python
def append_safe(x, bucket=None):
    """Append x to bucket but avoid shared default list."""
    ...
b1 = append_safe(1); b2 = append_safe(2)
assert b1 == [1] and b2 == [2]
```

3. **Clone 2-D list (shallow vs deep)**

```python
def shallow_clone_2d(m): ...
def deep_clone_2d(m): ...
m = [[1],[2]]
m2 = shallow_clone_2d(m); m3 = deep_clone_2d(m)
m[0].append(9)
assert m2[0] == [1,9] and m3[0] == [1]
```

### Core

4. **Freeze nested** (list/tuple/dict → immutables)

```python
def freeze(obj):
    """Convert lists to tuples, dicts to tuples of (key, frozen(value)) sorted by key."""
    ...
f = freeze({"a":[1,2], "b":{"x":1}})
assert isinstance(f, tuple) and ("a", (1,2)) in f
```

5. **Deep update (write-on-copy)**
   Merge dict `updates` into dict `base` without mutating `base`; nested dicts should merge recursively.

```python
def deep_merge(base, updates):
    ...
base = {"a":1,"b":{"x":1}}
u = {"b":{"y":2},"c":3}
merged = deep_merge(base,u)
assert base == {"a":1,"b":{"x":1}}
assert merged == {"a":1,"b":{"x":1,"y":2},"c":3}
```

6. **Sort safely** — Return a sorted copy by key; original list of dicts stays unchanged.

```python
def sort_users_by(users, key):
    ...
U = [{"name":"b","age":30},{"name":"a","age":20}]
out = sort_users_by(U,"name")
assert out[0]["name"]=="a" and U[0]["name"]=="b"
```

7. **Toggle flag in dict** — don’t mutate input

```python
def toggled(d, key):
    """Return a copy with boolean d[key] toggled (missing -> True)."""
    ...
assert toggled({"a":False},"a") == {"a":True}
```

### Challenge

8. **deep\_copy\_except(keys\_to\_skip)**
   Deep-copy a nested structure but **preserve references** for subtrees whose top-level keys are in `keys_to_skip` (for dicts).

```python
def deep_copy_except(obj, keys_to_skip=frozenset()):
    ...
src = {"cfg":{"x":1}, "cache":[1,2]}
out = deep_copy_except(src, {"cache"})
assert out["cfg"] is not src["cfg"] and out["cache"] is src["cache"]
```
